In [187]:
# Carregar Bibliotecas
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.tree import export_text

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split    
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn import datasets

import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb
from xgboost import XGBClassifier

In [188]:
# Carregar o dataset MNIST
digits = datasets.load_digits(n_class=10)

images=digits.images
targets=digits.target

#Redimensionar matrix
images=images.reshape(len(images),8*8)

X = digits.data
y = digits.target

# Criar nomes de características
n_pixels = X.shape[1]
feature_names = [f'pixel_{i}' for i in range(n_pixels)]

# Dividir o dataset em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(images, targets, test_size=0.2)

In [189]:
### DecisionTreeClassifier
#Melhores parâmetros encontrados: {'criterion': 'entropy', 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
#Precisão com os melhores parâmetros: 0.9763380758807589
#Acurácia: 98.33%

# Criar uma Decision Tree
# Configuração do DecisionTreeClassifier
clf = DecisionTreeClassifier(
    criterion='entropy',
    max_depth=None,
    max_features='sqrt',
    min_samples_leaf=1,
    min_samples_split=2,
    random_state=42
)

# Treinar a Decision Tree
clf.fit(X_train, y_train)

# Prever as classes para o conjunto de teste
y_pred_clf = clf.predict(X_test)

In [190]:
###XGBOOST
#Parametros extraídos do Gridsearch
#Melhores parâmetros encontrados: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 300, 'subsample': 0.8}
#Precisão com os melhores parâmetros: 0.9672861014324429

# Configurar o classificador XGBOOST
xgb = XGBClassifier(
    learning_rate=0.1,
    max_depth=3,
    n_estimators=100, 
    use_label_encoder=False, 
    eval_metric='mlogloss',
    colsample_bytree = 0.8,
    subsample= 0.8, 
    random_state=42
)

# Treinar o modelo usando GridSearchCV
xgb.fit(X_train, y_train)

# Prever as classes para o conjunto de teste com o melhor modelo encontrado
y_pred_xgb=xgb.predict(X_test)



/home/codespace/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [19:57:48] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [191]:
###RANDOM FOREST
#Parametros extraídos do Gridsearch
#Melhores parâmetros encontrados: {'criterion': 'entropy', 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
#Precisão com os melhores parâmetros: 0.9763380758807589

rf = RandomForestClassifier(
    n_estimators=300, 
    max_depth=None,
    max_features= 'sqrt', 
    criterion='entropy', 
    min_samples_leaf=1,
    min_samples_split= 2, 
    random_state=42
)

rf.fit(X_train, y_train)
y_pred_rf=rf.predict(X_test)

In [200]:
# Função para avaliar múltiplos modelos e retornar o melhor desempenho
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Função para avaliar múltiplos modelos e retornar o melhor desempenho
def evaluate_models(models_dict):
    # Armazenar os resultados de todos os modelos
    results_summary = []

    for model_name, (model, y_test, y_pred) in models_dict.items():
        # Avaliar o modelo
        accuracy = accuracy_score(y_test, y_pred)
        conf_matrix = confusion_matrix(y_test, y_pred)
        class_report = classification_report(y_test, y_pred)
        
        # Armazenar os resultados para comparação
        results_summary.append({
            'model_name': model_name,
            'accuracy': accuracy,
            'classification_report': class_report,
            'conf_matrix': conf_matrix
        })

    # Ordenar os resultados por acurácia em ordem decrescente
    results_summary.sort(key=lambda x: x['accuracy'], reverse=True)

    # Criar um array formatado com os resultados
    results_array = []
    for i, result in enumerate(results_summary):
        ranking = i + 1
        model_name = result['model_name']
        accuracy = result['accuracy'] * 100.0
        results_array.append(f"{ranking:<3} | {model_name:<25} | {accuracy:.1f}%")

    # Imprimir o array de resultados com cabeçalhos e bordas
    print("#   | Modelo                    | Acurácia")    
    print("----+---------------------------+---------") 
    for result in results_array:
        print(result)
        
    # Imprimir e visualizar os resultados dos modelos restantes
    for result in results_summary[3:]:
        model_name = result['model_name']
        print(f"Resultados para {model_name}:")
        print("Accuracy: %.2f%%" % (result['accuracy'] * 100.0))
        print("Relatório de Classificação:")
        print(result['classification_report'])
        
        # Visualizar a Matriz de Confusão
        plt.figure(figsize=(4, 3))
        sns.heatmap(result['conf_matrix'], annot=True, fmt='d', cmap='Blues', 
                    xticklabels=list(range(10)), yticklabels=list(range(10)))
        plt.title(f'Matriz de Confusão - {model_name}')
        plt.xlabel('Predição')
        plt.ylabel('Real')
        plt.show()

    return results_summary



In [201]:
################
#Avaliar Modelos
#Posso avaliar os modelos com entropy ou gini
################

models_dict = {
    "Decision Tree Classifier": (clf, y_test, y_pred_clf),
    "XGBoost": (xgb, y_test, y_pred_xgb),
    "Random Forest": (rf, y_test, y_pred_rf)
}

best_model_name, best_accuracy, results_summary = evaluate_models(models_dict)

#   | Modelo                    | Acurácia
----+---------------------------+---------
1   | Random Forest             | 97.8%
2   | XGBoost                   | 97.2%
3   | Decision Tree Classifier  | 78.9%
